In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from bs4 import BeautifulSoup
import time
import datetime
import re
import math

In [90]:
# Setup driver and fetch website
DRIVER_PATH = r"chromedriver"
DOWNLOAD_PATH = r"C:\Users\Dhruv\Documents\EPFO\downloads"
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_PATH,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
})
print("**** INITIALIZING DRIVER ****")
driver = webdriver.Chrome(executable_path=DRIVER_PATH,options=chrome_options)
url="https://unifiedportal-epfo.epfindia.gov.in/publicPortal/no-auth/misReport/home/loadEstSearchHome"
driver.get(url)
time.sleep(3)

**** INITIALIZING DRIVER ****


In [91]:
COMPANY_NAME = input("Enter company name: ")

# EST_ID = input("Establishment Code: ")

# est = driver.find_element_by_id("estCode")
# est.clear()
# est.send_keys(EST_ID)

name_of_establishment = driver.find_element_by_id("estName")
name_of_establishment.clear()
name_of_establishment.send_keys(COMPANY_NAME)

captacha = driver.find_element_by_id("captcha")
captacha.clear()
captacha_value = input(" **** Enter Captacha: ")
captacha.send_keys(captacha_value)

search_button = driver.find_element_by_id("searchEmployer")
search_button.click()

In [92]:
def row_count():
    # count the number of rows in the table
    table = driver.find_element_by_id("example")
    rows = table.find_elements(By.TAG_NAME, "tr")
    print("Number of rows in the table: ", len(rows))
    return len(rows)

In [93]:
def table_click(row):
    links = driver.find_elements("xpath", '//*[@id="example"]/tbody/tr['+str(row)+']/td[5]/a')
    if links:
        links[0].click()

In [51]:
import datetime
date = datetime.datetime.strptime("APR-22", "%b-%y")

In [83]:
date1 = datetime.datetime(2022,6,1)

In [53]:
print(date)

2022-04-01 00:00:00


In [49]:
print(date1)

2022-06-01 00:00:00


In [ ]:
row_len = row_count()

for row in range(1, row_len):
    table_click(row)
    time.sleep(5)
    create_tables(begin)
    begin = begin + 17
    payments_link = driver.find_element("xpath",'//*[@id="tablecontainer3"]/div/a/u')
    payments_link.click()
    time.sleep(5)

    driver.switch_to.window(driver.window_handles[1])
    pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
    pattern = r"(\d+)$"

    nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)
    if nos_pages_in_payments:
        number_of_pays = nos_pages_in_payments.group(1)
        print(number_of_pays)
    for nos in range(1,int(number_of_pays)+1):
        begin = payments_table(begin)
        row_number = begin + 1  
        next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
        next_button.click()
        time.sleep(1)
    row_color = '00FF00'
    fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')
    row = worksheet[row_number]

    for cell in row:
        cell.fill = fill
    time.sleep(5)
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)

workbook.save(filename=str(COMPANY_NAME)+".xlsx")

In [69]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.close()

In [ ]:
pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
pattern = r"(\d+)$"

nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)

if nos_pages_in_payments:
    number_of_pays = nos_pages_in_payments.group(1)
    print(number_of_pays)

In [ ]:
for nos in range(1,int(number_of_pays)+1):
    # payments_table(begin)
    begin = begin + 10
    next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
    next_button.click()
    # time.sleep(2)

In [6]:
total_records = driver.find_element("xpath",'//*[@id="collapseTwo"]/div[1]/div')

In [7]:
pattern = r"\d+"
matches = re.findall(pattern , total_records.text)
if matches:
    number = int(matches[0])
    print(number)

11


In [94]:
workbook = Workbook()
worksheet = workbook.active
begin = 1

In [54]:
def payments_table(begin)->int:
    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up')

    rows = table.find_all('tr')

    for row_index, row in enumerate(rows, start=begin):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            if col_index == 2:
                payment_date = datetime.datetime.strptime(cell_value,'%d-%b-%Y %H:%M:%S')
            if col_index == 4:
                wage_date = datetime.datetime.strptime(cell_value, '%b-%y')
                wage_month = wage_date.month
                wage_year = wage_date.year
                wage_day = wage_date.day
                if  wage_date >=  date1:
                    cell_value = wage_date.strftime("%b-%y")
                    worksheet.cell(row=row_index, column=col_index, value=cell_value)
                    next_month = (wage_month % 12) + 1
                    payment_due_date = datetime.datetime(wage_year, next_month, 15)
                    if payment_due_date < payment_date:
                        worksheet.cell(row=row_index, column=7,value="DELAYED")
                    else:
                        worksheet.cell(row=row_index, column=7,value="-")
                else:
                    begin -= 1
                    break
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
        begin += 1
    return begin-1

In [55]:
def create_tables(start_val):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    bold_font = Font(bold=True)

    table = soup.select_one('#tablecontainer3 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val,column=1,value="Validity Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val+1):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)


    table = soup.select_one('#tablecontainer4 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val + 6,column=1,value="Establishment Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 7):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    table = soup.select_one('#tablecontainer5 table')
    rows = table.find_all('tr')

    worksheet.cell(row=start_val + 12,column=1,value="Establishment Details-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 13):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            if cell_value == 'VERIFIED':
                fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            elif cell_value == 'REJECTED':
                fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
    
    table = soup.select_one('#tablecontainer12 table')
    rows = table.find_all('tr')

    worksheet.cell(row=start_val + 23,column=1,value="Additional Information-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 24):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)


    row_color = 'ADD8E6'
    fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')

    row_number = start_val + 32
    row = worksheet[row_number]

    for cell in row:
        cell.fill = fill

In [56]:
def heading(begin):
    bold_font = Font(bold=True)
    worksheet.cell(row=begin,column=1,value="TRRN").font = bold_font
    worksheet.cell(row=begin,column=2,value="Date of Credit").font = bold_font
    worksheet.cell(row=begin,column=3,value="Amount").font = bold_font
    worksheet.cell(row=begin,column=4,value="Wage Month").font = bold_font
    worksheet.cell(row=begin,column=5,value="No. of Employees").font = bold_font
    worksheet.cell(row=begin,column=6,value="ECR").font = bold_font
    worksheet.cell(row=begin,column=7,value="DELAY").font = bold_font

In [85]:
# number = math.ceil(number / 10)
# print(number)
# for i in range(1,number+1):
for i in range(1,2):
    row_len = 2
    for row in range(1, row_len):
        table_click(row)
        time.sleep(10)
        create_tables(begin)
        begin = begin + 33
        payments_link = driver.find_element("xpath",'//*[@id="tablecontainer3"]/div/a/u')
        payments_link.click()
        time.sleep(10)

        driver.switch_to.window(driver.window_handles[1])
        
        pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
        pattern = r"(\d+)$"
        nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)

        if nos_pages_in_payments:
            number_of_pays = nos_pages_in_payments.group(1)
            print("Number of payment pages:",number_of_pays)
            heading(begin)
        for nos in range(1,int(number_of_pays)+1):
            begin = payments_table(begin)
            row_number = begin + 1
            next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
            next_button.click()
        row_color = 'ADD8E6'
        fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')
        row = worksheet[row_number]

        for cell in row:
            cell.fill = fill
        time.sleep(10)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(3)
        begin = begin + 1

    # next = driver.find_element("xpath",'//*[@id="example_next"]')
    # next.click()
    # time.sleep(3)

Number of payment pages: 39


In [41]:
def header_2(begin):
    bold_font = Font(bold=True)
    worksheet.cell(row=begin,column=1,value="E_ID").font = bold_font
    worksheet.cell(row=begin,column=2,value="E_Name").font = bold_font
    worksheet.cell(row=begin,column=3,value="Address").font = bold_font
    worksheet.cell(row=begin,column=4,value="Office Name").font = bold_font

In [86]:
def same_pan_company(start_val)->int:
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up_wrapper table')

    rows = table.find_all('tr')
    
    for row_index, row in enumerate(rows, start=start_val):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
        start_val += 1

    return start_val

In [87]:
same_pan = driver.find_element("xpath",'//*[@id="tablecontainer11"]/div/a')
same_pan.click()
time.sleep(5)
driver.switch_to.window(driver.window_handles[1])
begin +=1 
bold_font = Font(bold=True)
worksheet.cell(row=begin,column=1,value="Establishments with same PAN:").font = bold_font
begin += 1
header_2(begin)
for i in range(1,3):
    begin = same_pan_company(begin)
    next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
    next_button.click()

In [96]:
driver.switch_to.window(driver.window_handles[1])

In [97]:
total_records = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')

pattern = r"\d+"
matches = re.findall(pattern , total_records.text)
if matches:
    number = int(matches[0])
    print(number)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="table_pop_up_info"]"}
  (Session info: chrome=114.0.5735.110)


In [ ]:
def table_click_2(row):
    E_ID = driver.find_elements("xpath", '//*[@id="table_pop_up"]/tbody/tr['+str(row)+']/td[1]/a')
    if E_ID:
        E_ID[0].click()
        return True

In [ ]:
def row_count_2():
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up_wrapper table')
    rows = table.find_all('tr')
    return len(rows)

In [ ]:
try:
    wb = load_workbook(r'C:\Users\Dhruv\Documents\EPFO\downloads\Payment Details.xlsx')
except Exception as e:
    print(f"error {e}")

In [88]:
workbook.save(filename=str(COMPANY_NAME)+".xlsx")

In [ ]:
same_pan = driver.find_element("xpath",'//*[@id="tablecontainer11"]/div/a')
same_pan.click()

In [ ]:
html_content = driver.page_source

soup = BeautifulSoup(html_content, 'html.parser')
table = soup.select_one('#tablecontainer3 table')

workbook = Workbook()
worksheet = workbook.active

rows = table.find_all('tr')

for row_index, row in enumerate(rows, start=1):
    columns = row.find_all('td')
    for col_index, column in enumerate(columns, start=1):
        cell_value = column.get_text()
        worksheet.cell(row=row_index, column=col_index, value=cell_value)

table = soup.select_one('#tablecontainer4 table')

rows = table.find_all('tr')

for row_index, row in enumerate(rows, start=8):
    columns = row.find_all('td')
    for col_index, column in enumerate(columns, start=1):
        cell_value = column.get_text()
        worksheet.cell(row=row_index, column=col_index, value=cell_value)

pan_status = driver.find_element("xpath",'//*[@id="tablecontainer5"]/div[2]/table/tbody/tr[2]/td[2]')

worksheet.cell(row=14,column=1,value="PAN STATUS:")
worksheet.cell(row=14,column=2,value=pan_status.text)

workbook.save(filename="epfo.xlsx")

In [ ]:
table_click(1)

In [34]:
same_pan = driver.find_element("xpath",'//*[@id="tablecontainer11"]/div/a')
same_pan.click()

driver.switch_to.window(driver.window_handles[1])

In [ ]:
# E_ID = driver.find_element("xpath",'//*[@id="table_pop_up"]/tbody/tr[1]/td[1]/a')
from selenium.common.exceptions import NoSuchElementException
try:
    element = driver.find_element_by_xpath('//*[@id="table_pop_up"]/tbody/tr[1]/td[1]/a')
    # Proceed with further actions on the element
    print(element.text)
except NoSuchElementException:
    # Handle the case when the element is not found
    print("Element not found")

In [38]:
a=1
worksheet.cell = (row=1,column=1,value="Establishments with same PAN:")
header_2(0)
for i in range(1,3):
    same_pan_company(a)
    next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
    a += 10


In [ ]:
def create_tables_2(start_val):
    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up_wrapper table')

    rows = table.find_all('tr')

    for row_index, row in enumerate(rows, start=start_val):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    table = soup.select_one('//*[@id="tbPopUp_4"]')

    rows = table.find_all('tr')

    for row_index, row in enumerate(rows, start=start_val + 6):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    pan_status = driver.find_element("xpath",'//*[@id="tablecontainer5"]/div[2]/table/tbody/tr[2]/td[2]')

    worksheet.cell(row=start_val + 13,column=1,value="PAN STATUS:")
    worksheet.cell(row=start_val + 13,column=2,value=pan_status.text)

    esic_code = driver.find_element("xpath", '//*[@id="tablecontainer12"]/table/tbody/tr[3]/td[2]')
    worksheet.cell(row=start_val + 14,column=1,value="ESIC Code:")
    worksheet.cell(row=start_val + 14,column=2,value=esic_code.text)

    row_color = '00FF00'
    fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')

    row_number = start_val + 15
    row = worksheet[row_number]

    for cell in row:
        cell.fill = fill

In [15]:
html_content = driver.page_source

soup = BeautifulSoup(html_content, 'html.parser')
table = soup.select_one('#tbPopUp_3')

workbook = Workbook()
worksheet = workbook.active

rows = table.find_all('tr')

for row_index, row in enumerate(rows, start=1):
    columns = row.find_all('td')
    for col_index, column in enumerate(columns, start=1):
        cell_value = column.get_text()
        worksheet.cell(row=row_index, column=col_index, value=cell_value)

table = soup.select_one('#tbPopUp_4')

rows = table.find_all('tr')

for row_index, row in enumerate(rows, start=8):
    columns = row.find_all('td')
    for col_index, column in enumerate(columns, start=1):
        cell_value = column.get_text()
        worksheet.cell(row=row_index, column=col_index, value=cell_value)

pan_status = driver.find_element("xpath",'//*[@id="tbPopUp_5"]/div[2]/table/tbody/tr[2]/td[2]')

worksheet.cell(row=14,column=1,value="PAN STATUS:")
worksheet.cell(row=14,column=2,value=pan_status.text)

esic_code = driver.find_element("xpath", '//*[@id="tbPopUp_12"]/table/tbody/tr[3]/td[2]')
worksheet.cell(row=15,column=1,value="ESIC Code:")
worksheet.cell(row=15,column=2,value=esic_code.text)

<Cell 'Sheet'.B15>

In [40]:
workbook.save(filename="test.xlsx") 